<h1>This code provides the functions used in the "Model.ipynb" code</h1>

In [ ]:
def TenTests(inputs, target):
    # returns the percentage error of each of the 10 tests performed
    import numpy as np
    from sklearn.model_selection import train_test_split
    import keras
    from keras.models import Sequential
    from keras.layers import Dense, Dropout
    from keras.callbacks import ModelCheckpoint
    percError = np.zeros((10, 1))
    filepath = 'best.hdf5'
    shape = inputs.shape[1]
    for x in range(10):
        # Split Data
        # Split test set (25%) from the rest of the data
        train1in, test_in, train1target, test_target = train_test_split(inputs, target, 
                                                                    test_size=1/4.0)
        # Split first 75% of data into training (50%) and validation (25%) sets
        train_in, val_in, train_target, val_target = train_test_split(train1in, train1target,
                                                                     test_size=1/3.0)

        # Create Model
        batch_size = 128
        epochs = 1000
        model = Sequential()
        model.add(Dense(30, activation = 'tanh', 
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros',
                        input_shape = (shape,)))
        model.add(Dropout(0.1))
        model.add(Dense(15, activation = 'tanh', 
                       kernel_initializer='glorot_uniform',
                       bias_initializer='zeros'))
        model.add(Dropout(0.1))
        model.add(Dense(10, activation = 'tanh',  
                       kernel_initializer='glorot_uniform',
                       bias_initializer='zeros'))
        model.add(Dense(5, activation = 'tanh', 
                       kernel_initializer='glorot_uniform',
                       bias_initializer='zeros'))
        model.add(Dense(1, activation = 'linear'))

        # Compile Model
        model.compile(loss = 'mean_absolute_percentage_error',
                     optimizer = 'Adam',
                     metrics = ['accuracy'])

        # Run model
        MC = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True)
        history = model.fit(train_in, train_target,
                 batch_size = batch_size,
                 epochs = epochs,
                 verbose = 0,
                 validation_data = (val_in, val_target),
                 callbacks = [MC])
        
        # load best model and get the percentage error
        model.load_weights("best.hdf5")
        scores = model.evaluate(test_in, test_target, verbose=0)
        percError[x] = scores[0]
        print('Test ', x, ' complete')
    return np.round(percError, 2)
